In [1]:
# Transformers installation
! pip install transformers datasets evaluate accelerate wandb
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


# Token classification

In [2]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/wVHdVlPScxA?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Token classification assigns a label to individual tokens in a sentence. One of the most common token classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This guide will show you how to:

1. Finetune [DistilBERT](https://huggingface.co/distilbert/distilbert-base-uncased) on the [WNUT 17](https://huggingface.co/datasets/wnut_17) dataset to detect new entities.
2. Use your finetuned model for inference.

<Tip>

To see all architectures and checkpoints compatible with this task, we recommend checking the [task-page](https://huggingface.co/tasks/token-classification).

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate seqeval
```

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Load WNUT 17 dataset

Start by loading the WNUT 17 dataset from the 🤗 Datasets library:

In [4]:
#from datasets import load_dataset

#wnut = load_dataset("wnut_17", data_dir="wnut_17", revision="refs/convert/parquet")
#gn = load_dataset("unimelb-nlp/wikiann", "gn")
#es = load_dataset("unimelb-nlp/wikiann", "es")

# Download from github: GUA-SPA
# clone repo
! git clone https://github.com/pln-fing-udelar/gua-spa-2023.git
! pip install conllu

Cloning into 'gua-spa-2023'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 229.92 KiB | 2.99 MiB/s, done.


In [5]:
import conllu

def read_conllu_file(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        data = f.read()
    return conllu.parse(data)

def conllu_to_dataset_format(parsed_data):
    dataset_format = {"tokens": [], "ner_tags": []}
    for sentence in parsed_data:
        tokens = []
        ner_tags = []
        for token in sentence:
            tokens.append(token["form"])
            # Assuming the NER tag is in the 'misc' field as a list of key=value pairs,
            # and one of them is 'ner=LABEL'. We need to adjust this based on the actual file format.
            if token['lemma'] is not None:
                ner_tag=token['lemma'].split('-')
                if 'ne' in ner_tag:
                    ner_tag = token['lemma']
                else:
                    ner_tag = ner_tag[0]
            ner_tags.append(ner_tag)
        dataset_format["tokens"].append(tokens)
        dataset_format["ner_tags"].append(ner_tags)
    return dataset_format


In [6]:
# Read an example conllu file
train_data = read_conllu_file("gua-spa-2023/gua_spa_train.txt")
# Convert to dataset format
train_dataset_format = conllu_to_dataset_format(train_data)
# Print the first example
print(train_dataset_format["tokens"][0])
print(train_dataset_format["ner_tags"][0])

['Aldana', "he'íva", 'umi', 'kits', 'ohóva', 'ha', 'oguahëva', 'opavave', "temimbo'épe", 'oñepyrû', 'mboyve', 'clase', 'pero', 'noñeguahëi', 'mbohapýha', 'ary', 'ohóvape', '.']
['ne-b-per', 'gn', 'gn', 'foreign', 'gn', 'gn', 'gn', 'gn', 'gn', 'gn', 'gn', 'es', 'es', 'gn', 'gn', 'gn', 'gn', 'other']


In [7]:
# Read an example conllu file
dev_data = read_conllu_file("gua-spa-2023/gua_spa_dev_gold.txt")
# Convert to dataset format
dev_dataset_format = conllu_to_dataset_format(dev_data)
# Print the first example
print(dev_dataset_format["tokens"][0])
print(dev_dataset_format["ner_tags"][0])

['Obligarle', 'alguien', 'pa', 'que', 'me', 'escriba', '?', 'No', 'señorito', 'ani', 'nde', 'kangy']
['es', 'es', 'es', 'es', 'es', 'es', 'other', 'es', 'es', 'gn', 'gn', 'gn']


In [8]:
# Read an example conllu file
test_data = read_conllu_file("gua-spa-2023/gua_spa_test_gold.txt")
# Convert to dataset format
test_dataset_format = conllu_to_dataset_format(test_data)
# Print the first example
print(test_dataset_format["tokens"][0])
print(test_dataset_format["ner_tags"][0])

['Igusto', "ñañe'ẽ", 'guaranime', 'ha', 'avei', 'japurahéi', 'umi', 'polka', 'ha', 'guarania']
['mix', 'gn', 'gn', 'gn', 'gn', 'gn', 'gn', 'foreign', 'gn', 'es']


In [9]:
from datasets import Dataset, DatasetDict

# Create Dataset objects from the parsed data
train_dataset = Dataset.from_dict(train_dataset_format)
dev_dataset = Dataset.from_dict(dev_dataset_format)
test_dataset = Dataset.from_dict(test_dataset_format)

# Create a DatasetDict
wnut = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset,
    "test": test_dataset
})

wnut

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1140
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 180
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 180
    })
})

Then take a look at an example:

In [10]:
wnut["train"][0]

{'tokens': ['Aldana',
  "he'íva",
  'umi',
  'kits',
  'ohóva',
  'ha',
  'oguahëva',
  'opavave',
  "temimbo'épe",
  'oñepyrû',
  'mboyve',
  'clase',
  'pero',
  'noñeguahëi',
  'mbohapýha',
  'ary',
  'ohóvape',
  '.'],
 'ner_tags': ['ne-b-per',
  'gn',
  'gn',
  'foreign',
  'gn',
  'gn',
  'gn',
  'gn',
  'gn',
  'gn',
  'gn',
  'es',
  'es',
  'gn',
  'gn',
  'gn',
  'gn',
  'other']}

Each number in `ner_tags` represents an entity. Convert the numbers to their label names to find out what the entities are:

In [11]:
# Get the unique tag names from the 'ner_tags' column
unique_tags = set()
for tags_list in wnut["train"]["ner_tags"]:
    unique_tags.update(tags_list)

label_list = sorted(list(unique_tags),
                    key=lambda x: (x.split('-')[-1]))
label_list

['es',
 'foreign',
 'gn',
 'ne-i-loc',
 'ne-b-loc',
 'mix',
 'ne-i-org',
 'ne-b-org',
 'other',
 'ne-b-per',
 'ne-i-per']

The letter that prefixes each `ner_tag` indicates the token position of the entity:

- `B-` indicates the beginning of an entity.
- `I-` indicates a token is contained inside the same entity (for example, the `State` token is a part of an entity like
  `Empire State Building`).
- `0` indicates the token doesn't correspond to any entity.

In [12]:
def assing_ner_id(example):
    example["ner_tags_name"] = example["ner_tags"]
    example["ner_tags"] = []
    for tag in example["ner_tags_name"]:
        example["ner_tags"].append(label_list.index(tag))
    return example

wnut = wnut.map(assing_ner_id)
wnut["train"][0], wnut["test"][0], wnut["validation"][0]

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

({'tokens': ['Aldana',
   "he'íva",
   'umi',
   'kits',
   'ohóva',
   'ha',
   'oguahëva',
   'opavave',
   "temimbo'épe",
   'oñepyrû',
   'mboyve',
   'clase',
   'pero',
   'noñeguahëi',
   'mbohapýha',
   'ary',
   'ohóvape',
   '.'],
  'ner_tags': [9, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 8],
  'ner_tags_name': ['ne-b-per',
   'gn',
   'gn',
   'foreign',
   'gn',
   'gn',
   'gn',
   'gn',
   'gn',
   'gn',
   'gn',
   'es',
   'es',
   'gn',
   'gn',
   'gn',
   'gn',
   'other']},
 {'tokens': ['Igusto',
   "ñañe'ẽ",
   'guaranime',
   'ha',
   'avei',
   'japurahéi',
   'umi',
   'polka',
   'ha',
   'guarania'],
  'ner_tags': [5, 2, 2, 2, 2, 2, 2, 1, 2, 0],
  'ner_tags_name': ['mix',
   'gn',
   'gn',
   'gn',
   'gn',
   'gn',
   'gn',
   'foreign',
   'gn',
   'es']},
 {'tokens': ['Obligarle',
   'alguien',
   'pa',
   'que',
   'me',
   'escriba',
   '?',
   'No',
   'señorito',
   'ani',
   'nde',
   'kangy'],
  'ner_tags': [0, 0, 0, 0, 0, 0, 8, 0, 0, 2, 2, 2],

In [13]:
ner = zip(wnut["train"][0]["tokens"], wnut["train"][0]["ner_tags"])
for item in ner:
  print(f"{item} => {label_list[item[1]]}")

('Aldana', 9) => ne-b-per
("he'íva", 2) => gn
('umi', 2) => gn
('kits', 1) => foreign
('ohóva', 2) => gn
('ha', 2) => gn
('oguahëva', 2) => gn
('opavave', 2) => gn
("temimbo'épe", 2) => gn
('oñepyrû', 2) => gn
('mboyve', 2) => gn
('clase', 0) => es
('pero', 0) => es
('noñeguahëi', 2) => gn
('mbohapýha', 2) => gn
('ary', 2) => gn
('ohóvape', 2) => gn
('.', 8) => other


## Preprocess

In [14]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/iY2AZYdZAr0?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The next step is to load a mBERT tokenizer to preprocess the `tokens` field:

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mmaguero/multilingual-bert-gn-base-cased") #"distilbert/distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [16]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'Al',
 '##dana',
 'he',
 "'",
 'í',
 '##va',
 'um',
 '##i',
 'kit',
 '##s',
 'o',
 '##h',
 '##ó',
 '##va',
 'ha',
 'og',
 '##uah',
 '##ë',
 '##va',
 'op',
 '##ava',
 '##ve',
 'temi',
 '##mbo',
 "'",
 'é',
 '##pe',
 'o',
 '##ñ',
 '##ep',
 '##yr',
 '##û',
 'm',
 '##boy',
 '##ve',
 'clase',
 'pero',
 'no',
 '##ñ',
 '##egu',
 '##ah',
 '##ë',
 '##i',
 'm',
 '##bo',
 '##ha',
 '##p',
 '##ý',
 '##ha',
 'ary',
 'o',
 '##h',
 '##ó',
 '##va',
 '##pe',
 '.',
 '[SEP]']

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function (see [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)).
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [17]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [18]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [20]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6332f45f8e491e35c06bf3132a159002147334db249d20e7be61bd952a1652dd
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [21]:
import evaluate

seqeval = evaluate.load("seqeval")

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [22]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [23]:
id2label = {
    0:'es',
    1:'foreign',
    2:'gn',
    3:'ne-i-loc',
    4:'ne-b-loc',
    5:'mix',
    6:'ne-i-org',
    7:'ne-b-org',
    8:'other',
    9:'ne-b-per',
    10:'ne-i-per'
}
label2id = {
    'es':0,
    'foreign':1,
    'gn':2,
    'ne-i-loc':3,
    'ne-b-loc':4,
    'mix':5,
    'ne-i-org':6,
    'ne-b-org':7,
    'other':8,
    'ne-b-per':9,
    'ne-i-per':10
}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [24]:
#!pip uninstall -y transformers
#!pip install transformers[tf]

In [25]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "mmaguero/multilingual-bert-gn-base-cased", # "distilbert/distilbert-base-uncased",
    num_labels=11, id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at mmaguero/multilingual-bert-gn-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
training_args = TrainingArguments(
    output_dir="gua-spa-2023-langid-ner-multilingual-bert-gn-base-cased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0.5, # 3 a 10 o más epochs
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit = 3,
    push_to_hub=False, # True para subir a HF
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate(tokenized_wnut["test"])

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [ ]:
trainer.push_to_hub()

<Tip>

For a more in-depth example of how to finetune a model for token classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [ ]:
text = "Golden State Warriors ha'e peteĩ equipo profesional de baloncesto USA pegua oĩva San Francisco-pe."
text_tokens = text.split()

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

classifier = pipeline("ner", model="mmaguero/wikiann-multilingual-bert-gn-base-cased", aggregation_strategy="max")

In [ ]:
classifier(text_tokens)

In [ ]:
wnut["test"][0]["tokens"], wnut["test"][0]["ner_tags"]

In [ ]:
classifier(wnut["test"][0]["tokens"])

In [ ]:
text = "Mañana voy a ir a Asunción y luego iré a Luque a la casa de mi tío Carlos."
text_tokens = text.split()
text_tokens, classifier(text_tokens)

In [ ]:
text = "Ko'ero aháta Paraguay-pe ha upé ahásata Luque-pe che tío Kálo ogápe."
text_tokens = text.split()
text_tokens, classifier(text_tokens)

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mmaguero/wikiann-multilingual-bert-gn-base-cased")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [ ]:
import torch
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("mmaguero/wikiann-multilingual-bert-gn-base-cased")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [ ]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
inputs, predicted_token_class